In [38]:
import pandas as pd
import numpy as np

#names=['Datum(UTC)','Nicht Erneuerbar (MW)', 'Erneuerbar (MW)', 'DE-LU Day Ahead EUR/MWh']
Wetter_Eki=pd.read_csv("Wetter/2024_Stunde_117_Ehrenkirchen.csv",sep=';',decimal=",",header=0,na_values='-').fillna(0)

In [40]:
#Stromfluss['Datum']=pd.to_datetime(Stromfluss['Datum von'],format='%d.%m.%Y %H:%M')
#Stromfluss['Datum']=Stromfluss['Datum'].dt.tz_localize('Europe/Berlin',ambiguous='infer')
#Stromfluss['Date']=Stromfluss['Datum'].dt.tz_convert('UTC')
Wetter_Eki['Datum']=pd.to_datetime(Wetter_Eki['Tag']+' '+Wetter_Eki['Stunde']+' +0100',format='%d.%m.%Y %H:%M %z',utc=True)
#Wetter_Eki['Datum']=Wetter_Eki['Datum'].dt.tz_localize('Europe/Berlin',ambiguous='infer')
Wetter_Eki=Wetter_Eki.set_index('Datum')

In [41]:
names=['Datum(UTC)','Nicht Erneuerbar (MW)', 'Erneuerbar (MW)', 'DE-LU Day Ahead EUR/MWh']
DayAheadPrices=pd.read_csv("Daten entsoe/energy-charts_Stromproduktion_und_Börsenstrompreise_in_Deutschland_2024.csv",skiprows=2,names=names)
DayAheadPrices['Datum']=pd.to_datetime(DayAheadPrices['Datum(UTC)'])
DayAheadPrices=DayAheadPrices.set_index('Datum')
preis_menge=DayAheadPrices
preis_menge[['AVG_TA200','AVG_WV200','MAX_WV200max','SUM_GS200']]=Wetter_Eki[['AVG_TA200','AVG_WV200','MAX_WV200max','SUM_GS200']]

In [42]:
preis_menge['AVG_WV200'].mean(),preis_menge['MAX_WV200max'].mean()

(1.5477345173041894, 3.3853483606557377)

In [43]:
preis_menge['Wind 6,5 m/s']=6.5*preis_menge['MAX_WV200max']/preis_menge['MAX_WV200max'].mean()

In [21]:
Windkraft_je_ms=pd.read_csv('Windanlagen_kw_pro_ms.csv',sep=';')
Windkraft_je_ms['max W/m²']=Windkraft_je_ms.apply(lambda row: pow(min(row['m/s'],15),3)*0.5*1.2, axis= 1)
Windkraft_je_ms['m/s cp']=Windkraft_je_ms['m/s']
Windkraft_je_ms.set_index('m/s',drop=True,inplace=True)
Windkraft_je_ms.head()

,V162-5.6,V162-6.2,V172-7.2,V172-7.2SO,EK100,1998_600kw,E-126,Kitepower_Hawk,Kitepower_Falcon,SkySails_PN-14,max W/m²,m/s cp
m/s,,,,,,,,,,,,
0.0,0,0,0,0,0.0,0,0,-7.0,-4.0,0.0,0.000,0.0
0.5,0,0,0,0,0.0,0,0,-7.0,-4.0,0.0,0.075,0.5
1.0,0,0,0,0,0.0,0,0,-6.5,-3.0,0.0,0.600,1.0
1.5,0,0,0,0,0.0,0,0,-6.0,-2.0,0.0,2.025,1.5
2.0,0,0,0,0,0.0,0,0,-5.5,-0.5,0.0,4.800,2.0


In [44]:
preis_menge

,Datum(UTC),Nicht Erneuerbar (MW),Erneuerbar (MW),DE-LU Day Ahead EUR/MWh,AVG_TA200,AVG_WV200,MAX_WV200max,SUM_GS200,"Wind 6,5 m/s"
Datum,,,,,,,,,
2023-12-31 23:00:00+00:00,2023-12-31T23:00+00:00,8378.54,41780.27,0.10,7.9,4.5,7.3,0.0,14.016283
2024-01-01 00:00:00+00:00,2024-01-01T00:00+00:00,8284.44,41524.81,0.01,8.1,4.6,10.4,0.0,19.968403
2024-01-01 01:00:00+00:00,2024-01-01T01:00+00:00,8282.03,41348.67,0.00,8.0,4.9,8.6,0.0,16.512333
2024-01-01 02:00:00+00:00,2024-01-01T02:00+00:00,8318.30,40365.41,-0.01,7.4,5.6,11.2,0.0,21.504434
2024-01-01 03:00:00+00:00,2024-01-01T03:00+00:00,8309.99,40235.53,-0.03,6.8,5.1,9.4,0.0,18.048364
...,...,...,...,...,...,...,...,...,...
2024-12-31 18:00:00+00:00,2024-12-31T18:00+00:00,21383.15,38916.54,67.77,6.4,2.7,6.5,0.0,12.480252
2024-12-31 19:00:00+00:00,2024-12-31T19:00+00:00,18557.03,40027.98,35.56,6.7,3.4,6.5,0.0,12.480252
2024-12-31 20:00:00+00:00,2024-12-31T20:00+00:00,15779.81,40653.81,15.70,5.4,2.5,7.7,0.0,14.784298
